# Vectorized Functions in Python

In [17]:
import numpy as np
from platform import python_version
python_version()

'3.6.10'

In [18]:
def lorenz(y, sigma, beta, rho):
    """The Lorenz ordinary differential equations.
    
    Arguments:
        y (array): Array of y values shape (..., 3).
        sigma (float): Lorenz parameter.
        beta (float): Lorenz parameter.
        rho (float): Lorenz parameter.
    
    Returns:
        dydt (tuple): Derivatives of y (dy/dt).
    """
    return (
        sigma * (y[1] - y[0]), 
        y[0] * (rho - y[2]) - y[1], 
        y[0] * y[1] - beta * y[2]
    )

# Lorenz system parameters
beta = 8 / 3
sigma = 10
rho = 28

In [19]:
# Test1 - Single function call
y0 = (-8, 8, 27)
assert lorenz(y0, sigma, beta, rho) == (160, -16, -136)

In [20]:
# Test 2 - Single function calls with for loop
def make_function_calls(f, y, n, *args, **kwargs):
    dy = np.empty_like(y)
    for i in range(n):
        dy[:, i] = f(y[:, i], *args, **kwargs)
    return dy

y = np.repeat(np.array((-8, 8, 27)).reshape(-1,1), 10, axis=1)
dy = make_function_calls(lorenz, y, 10, sigma=sigma, beta=beta, rho=rho)
assert dy.shape == y.shape
assert np.array_equiv(dy.T, (160, -16, -136))

In [21]:
# Test 3 - vectorized function call
y = np.repeat(np.array((-8, 8, 27)).reshape(-1,1), 10, axis=1)
dy = lorenz(y, sigma, beta, rho)
dy = np.vstack(dy)
assert dy.shape == y.shape
assert np.array_equiv(dy.T, (160, -16, -136))

## Speed tests

In [22]:
n = 10000
# Use n random y vectors
y = np.random.randn(3, n)

In [23]:
# Test 1 - Single function call
y0 = y[:, 0]
%timeit dy = lorenz(y0, sigma, beta, rho)

2.42 µs ± 72 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
# Test 2 - Single function calls with for loop
%timeit dy = make_function_calls(lorenz, y, n, sigma=sigma, beta=beta, rho=rho)

37.4 ms ± 465 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
# Test 3- Vectorized function calls
%timeit dy = lorenz(y, sigma, beta, rho)

58.2 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Recursive Calculation using a Generator

In [26]:
def lorenz_generator_discrete(y0, stepsize, sigma, beta, rho):
    y = np.array(y0)
    dy = np.empty(3)
    while True:
        dy[:] = lorenz(y, sigma, beta, rho)
        y = y + stepsize * dy
        yield y

def lorenz_generator_tuples(y0, stepsize, sigma, beta, rho):
    y = np.array(y0)
    dy = np.empty(3)
    while True:
        dy[:] = lorenz(y, sigma, beta, rho)
        y = y + stepsize * dy
        yield y[0], y[1], y[2]



# Lorenz system parameters
beta = 8 / 3
sigma = 10
rho = 28
y0 = (-8, 8, 27)
stepsize = 0.0001
gen = lorenz_generator_discrete(y0, stepsize, sigma, beta, rho)

In [27]:
y_data = np.zeros((100, 3))
for i in range(y_data.shape[0]):
    y_data[i] = next(gen)

In [28]:
gen = lorenz_generator_tuples(y0, stepsize, sigma, beta, rho)
next(gen)

(-7.984, 7.9984, 26.9864)

In [29]:
# Unfortunately, np.fromiter only works for 1D arrays
#np.fromiter(gen, float, count=10)